# Calibración de Probabilidades y Selección de Threshold con Costos Asimétricos

En problemas de clasificación binaria, la probabilidad predicha debe ser bien calibrada para que represente frecuencias reales. Cuando los costos de error son asimétricos (por ejemplo, el falso negativo es mucho más caro), además de calibrar, debemos optimizar el threshold para minimizar el costo esperado. Este notebook demuestra ambos aspectos de forma práctica y reproducible.

## 1. Introducción
- La calibración de probabilidades garantiza que, entre instancias con p=0.2, ~20% pertenezca a la clase positiva.
- Sin calibración, la selección de thresholds se vuelve poco confiable y puede subestimar/sobreestimar riesgos.
- Métricas clave: AUC ROC/PR (discriminación), Brier score (calibración), curvas de confiabilidad, matriz de confusión.
- Objetivo: entrenar un modelo, evaluar calibración, aplicar Platt scaling e Isotonic Regression, y optimizar threshold bajo costos asimétricos.

## 2. Caso práctico: diagnóstico de enfermedad grave (alto costo de FN)
- Contexto: un test algorítmico apoya diagnóstico de una enfermedad poco prevalente pero grave.
- Costo asimétrico: Falso Negativo (FN) implica no tratar a un paciente enfermo (costo alto). Falso Positivo (FP) implica derivar a prueba confirmatoria (costo moderado).
- Ejemplo de costos: C_FN = 50 unidades, C_FP = 1 unidad (ajustables).
- Meta: minimizar costo esperado con probabilidades calibradas y threshold óptimo.

## 3. Simulación de datos sintéticos
Generamos un dataset desbalanceado (prevalencia ~10%), con 20 variables, para simular el escenario médico.

In [ ]:
# Setup
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (roc_auc_score, roc_curve, precision_recall_curve,
                            average_precision_score, confusion_matrix, brier_score_loss,
                            precision_score, recall_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", context="talk")
RANDOM_STATE = 42
X, y = make_classification(n_samples=6000, n_features=20, n_informative=10, n_redundant=5,
                           n_clusters_per_class=2, weights=[0.9, 0.1], class_sep=1.2,
                           flip_y=0.01, random_state=RANDOM_STATE)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=RANDOM_STATE
)
print('Prevalencia train:', y_train.mean().round(3), '| Prevalencia test:', y_test.mean().round(3))


## 4. Entrenamiento del modelo (Logistic Regression)
Entrenamos una logística con estandarización. Guardamos probabilidades crudas (sin calibrar).

In [ ]:
base_clf = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(max_iter=1000, class_weight=None, random_state=RANDOM_STATE))
])
base_clf.fit(X_train, y_train)
p_base = base_clf.predict_proba(X_test)[:,1]
print('AUC (base, sin calibrar):', roc_auc_score(y_test, p_base).round(3))
print('Brier (base):', brier_score_loss(y_test, p_base).round(3))


## 5. Métricas iniciales y matriz de confusión (threshold 0.5)
Calculamos precisión, recall, F1 y la matriz de confusión con threshold estándar 0.5.

In [ ]:
def plot_confusion(cm, labels=('Neg','Pos'), title='Matriz de confusión'):
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=labels, yticklabels=labels)
    plt.ylabel('Real'); plt.xlabel('Predicho'); plt.title(title); plt.tight_layout(); plt.show()

thr_default = 0.5
y_pred_05 = (p_base >= thr_default).astype(int)
cm_05 = confusion_matrix(y_test, y_pred_05)
plot_confusion(cm_05, title=f'Matriz de confusión (thr={thr_default})')
print('Precision:', precision_score(y_test, y_pred_05).round(3))
print('Recall:', recall_score(y_test, y_pred_05).round(3))
print('F1:', f1_score(y_test, y_pred_05).round(3))


## 6. ROC y PR Curve
Las curvas ROC/PR evalúan la capacidad de discriminación. En datasets desbalanceados, el AUPRC (Average Precision) aporta contexto adicional al AUC ROC.

In [ ]:
# ROC
fpr, tpr, roc_th = roc_curve(y_test, p_base)
auc_roc = roc_auc_score(y_test, p_base)
plt.figure(figsize=(6,5)); plt.plot(fpr, tpr, label=f'AUC={auc_roc:.3f}')
plt.plot([0,1],[0,1],'k--', alpha=0.6)
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve (base)'); plt.legend(); plt.tight_layout(); plt.show()
# PR
prec, rec, pr_th = precision_recall_curve(y_test, p_base)
auprc = average_precision_score(y_test, p_base)
plt.figure(figsize=(6,5)); plt.plot(rec, prec, label=f'AP={auprc:.3f}')
plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('PR Curve (base)'); plt.legend(); plt.tight_layout(); plt.show()


## 7. Calibración: Platt scaling e Isotonic Regression
Usamos CalibratedClassifierCV sobre el clasificador base (validez out-of-sample vía validación cruzada). Comparamos Brier score y curvas de confiabilidad.

In [ ]:
from sklearn.base import clone
platt = CalibratedClassifierCV(estimator=clone(base_clf), method='sigmoid', cv=5)
platt.fit(X_train, y_train)
p_platt = platt.predict_proba(X_test)[:,1]
iso = CalibratedClassifierCV(estimator=clone(base_clf), method='isotonic', cv=5)
iso.fit(X_train, y_train)
p_iso = iso.predict_proba(X_test)[:,1]
print('AUC base:', roc_auc_score(y_test, p_base).round(3))
print('AUC platt:', roc_auc_score(y_test, p_platt).round(3))
print('AUC iso:  ', roc_auc_score(y_test, p_iso).round(3))
print('Brier base:', brier_score_loss(y_test, p_base).round(3))
print('Brier platt:', brier_score_loss(y_test, p_platt).round(3))
print('Brier iso:  ', brier_score_loss(y_test, p_iso).round(3))
from sklearn.calibration import calibration_curve as _cc
def plot_reliability(y_true, probs_list, labels, n_bins=10, title='Curva de confiabilidad'):
    plt.figure(figsize=(6,5))
    for p, lab in zip(probs_list, labels):
        frac_pos, mean_pred = _cc(y_true, p, n_bins=n_bins, strategy='uniform')
        plt.plot(mean_pred, frac_pos, marker='o', label=lab)
    plt.plot([0,1],[0,1],'k--', alpha=0.6)
    plt.xlabel('Probabilidad predicha promedio')
    plt.ylabel('Fracción positiva')
    plt.title(title)
    plt.legend(); plt.tight_layout(); plt.show()
plot_reliability(y_test, [p_base, p_platt, p_iso], ['Base','Platt','Isotónica'])


## 8. Análisis de thresholds y costo esperado
Definimos: Costo = C_FP*FP + C_FN*FN. Exploramos thresholds y elegimos el que minimiza el costo (usamos probabilidades isotónicas).

In [ ]:
def cost_summary(y_true, p, thr, c_fp=1.0, c_fn=50.0):
    y_hat = (p >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    total = c_fp*fp + c_fn*fn
    return {'thr':thr, 'fp':int(fp), 'fn':int(fn), 'tp':int(tp), 'tn':int(tn), 'cost': float(total)}
thresholds = np.linspace(0.01, 0.99, 99)
df_cost_iso = pd.DataFrame([cost_summary(y_test, p_iso, t, c_fp=1.0, c_fn=50.0) for t in thresholds])
df_cost_iso.head()


In [ ]:
plt.figure(figsize=(7,5))
plt.plot(df_cost_iso['thr'], df_cost_iso['cost'], label='Costo total (Isotónica)')
plt.axvline(0.5, color='grey', linestyle='--', alpha=0.6, label='thr=0.5')
t_opt = df_cost_iso.loc[df_cost_iso['cost'].idxmin(), 'thr']
c_opt = df_cost_iso['cost'].min()
plt.axvline(t_opt, color='crimson', linestyle='--', label=f'thr óptimo={t_opt:.3f}')
plt.xlabel('Threshold'); plt.ylabel('Costo total'); plt.title('Costo vs Threshold')
plt.legend(); plt.tight_layout(); plt.show()
print(f'Threshold óptimo (mínimo costo): {t_opt:.3f} | Costo: {c_opt:.1f}')


### Matriz de confusión y métricas: thr=0.5 vs thr óptimo
Comparamos desempeño con probabilidades isotónicas.

In [ ]:
y_pred_iso_05 = (p_iso >= 0.5).astype(int)
y_pred_iso_opt = (p_iso >= t_opt).astype(int)
cm_iso_05 = confusion_matrix(y_test, y_pred_iso_05)
cm_iso_opt = confusion_matrix(y_test, y_pred_iso_opt)
plot_confusion(cm_iso_05, title='Isotónica - thr=0.5')
plot_confusion(cm_iso_opt, title=f'Isotónica - thr óptimo={t_opt:.3f}')
print('Precision@0.5:', precision_score(y_test, y_pred_iso_05).round(3), '| Recall@0.5:', recall_score(y_test, y_pred_iso_05).round(3))
print('Precision@opt:', precision_score(y_test, y_pred_iso_opt).round(3), '| Recall@opt:', recall_score(y_test, y_pred_iso_opt).round(3))


## 9. Conclusión
- La calibración (Platt/Isotónica) reduce el Brier score y mejora la confiabilidad.
- Con costos asimétricos, es subóptimo usar thr=0.5; debe elegirse el threshold que minimiza el costo esperado en el conjunto de validación/test.
- Flujo recomendado: (1) entrenar, (2) calibrar probabilidades, (3) cuantificar costo FN/FP, (4) optimizar threshold según costo, (5) monitorear en producción.